## Uncertainty Quantification for fine-tuned models

Let's repeat the exercise for the case where you are fine-tuning an existing model. 

Before anything, you must download the PET-MAD checkpoint that will be used in this tutorial (you can also copy from another directory, if you've downloaded the model already).

```bash
wget https://huggingface.co/lab-cosmo/pet-mad/resolve/v1.0.2/models/pet-mad-v1.0.2.ckpt
```

In [ ]:
import ase.build
import ase.io
import matplotlib.pyplot as plt
import numpy as np

from ase.calculators.emt import EMT

calculator = EMT()
structure = ase.build.bulk("Al", "fcc", cubic=True)

Same as before, we generate 1000 structures and compute the energies using the effective medium potential (EMT), then save the datasets:

In [ ]:
structures = []
for i in range(1000):
    s = structure.copy()
    s.rattle(0.3, seed=i)
    s.calc = calculator
    s.info["energy"] = s.get_potential_energy()
    s.arrays["forces"] = s.get_forces()
    s.info["stress"] = s.get_stress(voigt=False)
    s.calc = None
    structures.append(s)

ase.io.write("dataset.xyz", structures[:50])
ase.io.write("evaluation.xyz", structures[50:])

You can now launch the model fine-tuning, LLPR training, and evaluation processes with the `metatrain` CLI.

Before running the below commands, **take a moment to study the YAML files being used** and make sure you understand the options being passed to `metatrain`. *Ask questions!*

```bash
mtt train options.yaml
mtt train options-llpr.yaml -o model-llpr.pt
mtt eval model-llpr.pt eval.yaml -b 20
```

We plot the spread of actual errors vs. the predicted errors from the LLPR formalism, now for the model fine-tuned from PET-MAD:

In [ ]:
# Load the structures
structures = ase.io.read("evaluation.xyz", ":")
evaluated_structures = ase.io.read("output.xyz", ":")

# Obtain the predictions and uncertainties from the evaluated structures
predicted_energies = [s.get_potential_energy() for s in evaluated_structures]
true_energies = [s.get_potential_energy() for s in structures]
errors = np.abs(np.array(predicted_energies) - np.array(true_energies))
uncertainties = [s.info["energy_uncertainty"] for s in evaluated_structures]
ensemble_uncertainties = [s.info["energy_ensemble"].std() for s in evaluated_structures]

quantile_lines = [0.00916, 0.10256, 0.4309805, 1.71796, 2.5348, 3.44388]
min_val_x = np.min(uncertainties)
max_val_x = np.max(uncertainties)
min_val_y = np.min(errors)
max_val_y = np.max(errors)

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
# Plot using normal uncertainties
ax[0].plot([min_val_x, max_val_x], [min_val_x, max_val_x], "k--", lw=0.75)
for factor in quantile_lines:
    ax[0].plot([min_val_x, max_val_x], [factor * min_val_x, factor * max_val_x], "k:", lw=0.75)
ax[0].scatter(uncertainties, errors)
ax[0].set_xscale("log")
ax[0].set_yscale("log")
ax[0].set_xlabel("Predicted energy uncertainty (eV)")
ax[0].set_ylabel("Absolute error in predicted energy (eV)")
ax[0].set_title("From LLPR")
ax[0].grid()
# Plot using ensemble uncertainties
ax[1].plot([min_val_x, max_val_x], [min_val_x, max_val_x], "k--", lw=0.75)
for factor in quantile_lines:
    ax[1].plot([min_val_x, max_val_x], [factor * min_val_x, factor * max_val_x], "k:", lw=0.75)
ax[1].scatter(ensemble_uncertainties, errors)
ax[1].set_xscale("log")
ax[1].set_yscale("log")
ax[1].set_xlabel("Predicted energy ensemble std (eV)")
ax[1].set_ylabel("Absolute error in predicted energy (eV)")
ax[1].set_title("From LLPR-derived shallow ensemble")
ax[1].grid()
